Evaluate LLM Based applications

visualizers and Debuggers

language models and chains them self to evaluate other language models

In [1]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 7


In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings

In [5]:
file = "OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

In [8]:
embeddings=OpenAIEmbeddings()

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_24532/2276573386.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings()


In [10]:
index = VectorstoreIndexCreator(
    embedding = embeddings,
    vectorstore_cls = DocArrayInMemorySearch
).from_loaders([loader])

/Users/apple/anaconda3/envs/enve/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [14]:
llm = ChatOpenAI(temperature=0.0)
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = index.vectorstore.as_retriever(),
    verbose = True,
    # chain_type_kwargs = {
    #     "document_seperator": "<<<<>>>>"
    # }
)

1. Comeup with datapoints that we think is the good examples by looking at some of the data

    and comeup with example question and example groundtruth answers that we can later use to evaluate. 

    This method takes time to look through each example and does not really scale well.

2. Automate it using Language models.using QAGenerateChain from langchain :- It will create a question answer pair from each document.

    It will do this using Language model itself, we need to create this chain by passing in the Chat OpenAI language model.

In [15]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\r\n\r\nImported.")

In [16]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

In [18]:
examples = [
    {
        "query": "Do cozy Comfort Pullovers set have a side pocket?",
        "answer": "Yes" 
    },
    {
        "query": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [19]:
from langchain.evaluation.qa import QAGenerateChain

In [20]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [21]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc":t} for t in data[:5]]
)

/Users/apple/anaconda3/envs/enve/lib/python3.10/site-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [23]:
new_examples[0]

{'qa_pairs': {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords is 1 lb. 1 oz. per pair."}}

In [24]:
examples += new_examples

In [25]:
examples

[{'query': 'Do cozy Comfort Pullovers set have a side pocket?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'qa_pairs': {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
   'answer': "The approximate weight of the Women's Campside Oxfords is 1 lb. 1 oz. per pair."}},
 {'qa_pairs': {'query': 'What are the dimensions of the small and medium sizes for the Recycled Waterhog Dog Mat in Chevron Weave?',
   'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'}},
 {'qa_pairs': {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece offer according to the document?",
   'answer': "The swimsuit features bright colors, ruffles, and exclusive whimsical prints. It is made of four-way-stretch and chlorine-resistant fabric that retains its shape and resists snags. The UPF 

In [29]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do cozy Comfort Pullovers set have a side pocket?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do cozy Comfort Pullovers set have a side pocket?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- P

'Yes, the Cozy Comfort Pullover Set, Stripe does have side pockets on the pants.'

Here we can see only answer, but what are the documents that it retrieves? and what is the actual prompt going to LLM.

what are the intermediate results?

Sometimes we get wrong result is return, its not LLM that is messing up but can be retrieval that is messing up.

So looking at what the question is and what the answer is will help debug whats going on.

In [28]:
import langchain
langchain.debug = True